In [2]:
#!/usr/bin/env python3
"""
Scan physical constants for quarter-dyadic ladder patterns
Test whether constants fit: φ^(2k+1) / q^(2m+1) where q ∈ {2,3,5,7,11,...}
"""

import mpmath as mp
import numpy as np
from typing import Dict, List, Tuple, Optional
import itertools

# Set high precision
mp.dps = 50

# Physical constants (CODATA 2018 values)
PHYSICAL_CONSTANTS = {
    'alpha_inv': mp.mpf('137.035999084'),  # Fine structure constant^-1
    'alpha': mp.mpf('0.0072973525693'),    # Fine structure constant
    'electron_mass': mp.mpf('9.1093837015e-31'),  # kg
    'proton_mass': mp.mpf('1.67262192369e-27'),   # kg
    'mu_ratio': mp.mpf('1836.15267343'),   # Proton/electron mass ratio
    'planck_h': mp.mpf('6.62607015e-34'),  # Planck constant
    'hbar': mp.mpf('1.054571817e-34'),     # Reduced Planck constant
    'c': mp.mpf('299792458'),              # Speed of light m/s
    'G': mp.mpf('6.67430e-11'),            # Gravitational constant
    'k_B': mp.mpf('1.380649e-23'),         # Boltzmann constant
    'e_charge': mp.mpf('1.602176634e-19'), # Elementary charge
    'epsilon_0': mp.mpf('8.8541878128e-12'), # Vacuum permittivity
    'mu_0': mp.mpf('1.25663706212e-6'),    # Vacuum permeability
    'R_inf': mp.mpf('10973731.568160'),    # Rydberg constant
    'a_0': mp.mpf('5.29177210903e-11'),    # Bohr radius
    'sigma_sb': mp.mpf('5.670374419e-8'),  # Stefan-Boltzmann constant
}

# Mathematical constants
MATH_CONSTANTS = {
    'pi': mp.pi,
    'e': mp.e,
    'phi': (1 + mp.sqrt(5)) / 2,  # Golden ratio
    'sqrt2': mp.sqrt(2),
    'sqrt3': mp.sqrt(3),
    'sqrt5': mp.sqrt(5),
    'euler_gamma': mp.euler,
    'catalan': mp.catalan,
}

def generate_quarter_dyadic_values(max_k: int = 10, max_m: int = 5) -> List[Tuple[mp.mpf, int, int, int]]:
    """
    Generate quarter-dyadic ladder values: φ^(2k+1) / q^(2m+1)
    Returns list of (value, k, m, q)
    """
    phi = MATH_CONSTANTS['phi']
    small_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23]

    values = []

    for k in range(max_k + 1):
        for m in range(max_m + 1):
            for q in small_primes:
                if q**(2*m + 1) <= 1e12:  # Keep denominators reasonable
                    numerator = phi**(2*k + 1)
                    denominator = q**(2*m + 1)
                    value = numerator / denominator
                    values.append((value, k, m, q))

    return sorted(values, key=lambda x: float(x[0]))

def find_best_matches(target: mp.mpf, tolerance: float = 1e-3) -> List[Tuple[float, int, int, int, str]]:
    """
    Find quarter-dyadic values that match target within tolerance
    """
    quarter_dyadic_values = generate_quarter_dyadic_values()

    matches = []
    for value, k, m, q in quarter_dyadic_values:
        error = abs(value - target) / abs(target)
        if error < tolerance:
            pattern = f"φ^{2*k+1}/{q}^{2*m+1}"
            matches.append((float(error), k, m, q, pattern))

    return sorted(matches, key=lambda x: x[0])

def test_scaling_relationships(target: mp.mpf, name: str) -> List[Tuple[float, str]]:
    """
    Test various scaling relationships with mathematical constants
    """
    phi = MATH_CONSTANTS['phi']
    pi = MATH_CONSTANTS['pi']
    e = MATH_CONSTANTS['e']
    sqrt2 = MATH_CONSTANTS['sqrt2']
    sqrt3 = MATH_CONSTANTS['sqrt3']

    relationships = [
        # Power relationships
        (target / phi, f"{name}/φ"),
        (target / phi**2, f"{name}/φ²"),
        (target / phi**3, f"{name}/φ³"),
        (target * phi, f"{name}×φ"),
        (target * phi**2, f"{name}×φ²"),

        # Pi relationships
        (target / pi, f"{name}/π"),
        (target * pi, f"{name}×π"),
        (target / (4*pi), f"{name}/(4π)"),
        (target * 4*pi, f"{name}×4π"),

        # E relationships
        (target / e, f"{name}/e"),
        (target * e, f"{name}×e"),

        # Combined relationships
        (target / (phi * pi), f"{name}/(φπ)"),
        (target * phi / pi, f"{name}×φ/π"),
        (target * pi / phi, f"{name}×π/φ"),

        # Square root relationships
        (target * sqrt2, f"{name}×√2"),
        (target / sqrt2, f"{name}/√2"),
        (target * sqrt3, f"{name}×√3"),
        (target / sqrt3, f"{name}/√3"),

        # Logarithmic relationships
        (mp.log(target), f"ln({name})"),
        (mp.exp(target), f"exp({name})"),
    ]

    quarter_dyadic_values = generate_quarter_dyadic_values(max_k=8, max_m=4)

    best_matches = []
    for scaled_value, description in relationships:
        for qd_value, k, m, q in quarter_dyadic_values:
            error = abs(scaled_value - qd_value) / abs(qd_value)
            if error < 0.01:  # 1% tolerance
                pattern = f"φ^{2*k+1}/{q}^{2*m+1}"
                match_desc = f"{description} ≈ {pattern}"
                best_matches.append((float(error), match_desc))

    return sorted(best_matches, key=lambda x: x[0])[:5]  # Top 5 matches

def comprehensive_constant_scan():
    """
    Scan all physical constants for quarter-dyadic patterns
    """
    print("QUARTER-DYADIC LADDER PATTERN SCANNER")
    print("="*60)
    print("Testing whether physical constants fit: φ^(2k+1) / q^(2m+1)")
    print(f"Using {mp.dps} decimal places precision")
    print()

    # Test each physical constant
    all_results = {}

    for const_name, const_value in PHYSICAL_CONSTANTS.items():
        print(f"Testing {const_name} = {mp.nstr(const_value, 12)}")
        print("-" * 50)

        # Direct matches
        direct_matches = find_best_matches(const_value, tolerance=0.01)
        if direct_matches:
            print("Direct quarter-dyadic matches:")
            for error, k, m, q, pattern in direct_matches[:3]:
                print(f"  {pattern}: error = {error:.2e} ({error*100:.4f}%)")

        # Scaling relationships
        scaling_matches = test_scaling_relationships(const_value, const_name)
        if scaling_matches:
            print("\nBest scaling relationships:")
            for error, description in scaling_matches:
                print(f"  {description}: error = {error:.2e} ({error*100:.4f}%)")

        all_results[const_name] = {
            'direct': direct_matches[:3],
            'scaling': scaling_matches[:3]
        }

        print()

    # Special tests for known relationships
    print("SPECIAL RELATIONSHIP TESTS")
    print("="*40)

    # Test alpha^-1 = 101(φ^(π-2) - √3)
    alpha_inv = PHYSICAL_CONSTANTS['alpha_inv']
    phi = MATH_CONSTANTS['phi']
    pi = MATH_CONSTANTS['pi']
    sqrt3 = MATH_CONSTANTS['sqrt3']

    predicted_alpha_inv = 101 * (phi**(pi - 2) - sqrt3)
    alpha_error = abs(alpha_inv - predicted_alpha_inv) / alpha_inv
    print(f"α⁻¹ = 101(φ^(π-2) - √3)")
    print(f"Predicted: {mp.nstr(predicted_alpha_inv, 12)}")
    print(f"Actual:    {mp.nstr(alpha_inv, 12)}")
    print(f"Error:     {float(alpha_error):.2e} ({float(alpha_error)*100:.4f}%)")
    print()

    # Test mu = proton/electron mass ratio relationship
    mu = PHYSICAL_CONSTANTS['mu_ratio']
    # Test if mu relates to quarter-dyadic pattern
    mu_matches = find_best_matches(mu, tolerance=0.001)
    print(f"Proton/electron mass ratio μ = {mp.nstr(mu, 12)}")
    if mu_matches:
        print("Quarter-dyadic matches:")
        for error, k, m, q, pattern in mu_matches[:3]:
            print(f"  {pattern}: error = {error:.2e} ({error*100:.4f}%)")

    # Test mu in terms of Fibonacci decimal
    fib_decimal = mp.mpf('0.412037') # Fibonacci word as decimal
    mu_fib_test = 3**6 / fib_decimal
    mu_fib_error = abs(mu - mu_fib_test) / mu
    print(f"3⁶/Fib_decimal = {mp.nstr(mu_fib_test, 12)}")
    print(f"Error: {float(mu_fib_error):.2e} ({float(mu_fib_error)*100:.4f}%)")
    print()

    # Summary of best findings
    print("SUMMARY OF BEST QUARTER-DYADIC MATCHES")
    print("="*45)

    best_overall = []
    for const_name, results in all_results.items():
        for error, k, m, q, pattern in results['direct']:
            if error < 0.01:  # Less than 1% error
                best_overall.append((error, const_name, pattern))

        for error, description in results['scaling']:
            if error < 0.01:
                best_overall.append((error, const_name, description))

    best_overall.sort(key=lambda x: x[0])

    if best_overall:
        print("Constants with quarter-dyadic patterns (< 1% error):")
        for error, const_name, pattern in best_overall[:10]:
            print(f"  {const_name}: {pattern} (error: {error*100:.4f}%)")
    else:
        print("No strong quarter-dyadic patterns found within 1% tolerance.")
        print("This could mean:")
        print("1. The pattern is more subtle than expected")
        print("2. Additional scaling factors are needed")
        print("3. The pattern applies to different combinations of constants")

    return all_results

def test_constant_ratios():
    """
    Test ratios between constants for quarter-dyadic patterns
    """
    print("\nTESTING CONSTANT RATIOS")
    print("="*30)

    const_names = list(PHYSICAL_CONSTANTS.keys())
    phi = MATH_CONSTANTS['phi']

    best_ratio_matches = []

    for i, name1 in enumerate(const_names):
        for name2 in const_names[i+1:]:
            val1 = PHYSICAL_CONSTANTS[name1]
            val2 = PHYSICAL_CONSTANTS[name2]

            ratio = val1 / val2
            matches = find_best_matches(ratio, tolerance=0.01)

            if matches:
                error, k, m, q, pattern = matches[0]
                if error < 0.005:  # 0.5% threshold
                    ratio_desc = f"{name1}/{name2}"
                    best_ratio_matches.append((float(error), ratio_desc, pattern))

    best_ratio_matches.sort(key=lambda x: x[0])

    if best_ratio_matches:
        print("Best ratio matches:")
        for error, ratio_desc, pattern in best_ratio_matches[:5]:
            print(f"  {ratio_desc} ≈ {pattern} (error: {error*100:.3f}%)")
    else:
        print("No strong patterns found in constant ratios")

if __name__ == "__main__":
    results = comprehensive_constant_scan()
    test_constant_ratios()

    print(f"\n{'='*60}")
    print("ANALYSIS COMPLETE")
    print("If multiple constants show quarter-dyadic patterns,")
    print("this supports the fundamental nature of φ-based organization.")

QUARTER-DYADIC LADDER PATTERN SCANNER
Testing whether physical constants fit: φ^(2k+1) / q^(2m+1)
Using 50 decimal places precision

Testing alpha_inv = 137.035999084
--------------------------------------------------

Best scaling relationships:
  alpha_inv/(4π) ≈ φ^15/5^3: error = 6.44e-04 (0.0644%)
  alpha_inv/e ≈ φ^15/3^3: error = 2.09e-03 (0.2095%)
  alpha_inv/(4π) ≈ φ^9/7^1: error = 4.23e-03 (0.4232%)
  alpha_inv×√2 ≈ φ^15/7^1: error = 5.44e-03 (0.5435%)

Testing alpha = 0.0072973525693
--------------------------------------------------

Best scaling relationships:
  alpha×√3 ≈ φ^1/2^7: error = 1.19e-04 (0.0119%)
  exp(alpha) ≈ φ^5/11^1: error = 8.66e-04 (0.0866%)
  alpha/φ³ ≈ φ^7/7^5: error = 2.81e-03 (0.2807%)
  alpha×φ ≈ φ^11/7^5: error = 2.81e-03 (0.2807%)
  alpha/φ ≈ φ^9/7^5: error = 2.81e-03 (0.2807%)

Testing electron_mass = 9.1093837015e-31
--------------------------------------------------

Best scaling relationships:
  exp(electron_mass) ≈ φ^5/11^1: error = 8.13e-03 (0.